In [2]:
# Import modules/packages
import os
import numpy as np

# Import third-party modules/packages
import utils
from utils import File, Command, NiiFile

In [3]:
class ReconMRtrix (object):
    '''class doc-string'''
    
    nii_file: NiiFile = ""
    bval: File = ""
    bvec: File = ""
    json_file: File = ""
    log: LogFile = ""
    
    def __init__(self,
                nii_file: NiiFile,
                 bval: File,
                 bvec: File,
                 log: LogFile,
                 json_file: File = "") -> None:
        '''doc-string'''
        self.nii_file: NiiFile = nii_file
        self.bval: File = bval
        self.bvec: File = bvec
        self.log: LogFile = log
        if json_file:
            self.json_file: File = json_file
        else:
            self.json_file = ""
        
        
    class Mif(NiiFile):
        '''class doc-string'''
        
        def __init__(self,file):
            '''doc-string'''
            self.file = file
            NiiFile.__init__(self,self.file)
    
    def nifti_to_mif(nii_file,bval,bvec,json_file=""):
        '''doc-string'''
        if not nii_file:
            nii_file = self.nii_file
        if not bval:
            bval = self.bval
        if not bvec:
            bvec = self.bvec
        if not json_file:
            json_file = self.json_file
        
        mif_file: Mif = nii_file.abs_path()
        mif_file = mif_file.rm_ext() + ".mif"
        
        mr_convert = Command("mrconvert")
        if json_file:
            mr_convert.cmd_list.append("-json_import")
            mr_convert.cmd_list.append(f"{nii_file.file}")
        mr_convert.cmd_list.append("-fslgrad")
        mr_convert.cmd_list.append(f"{bvec.file}")
        mr_convert.cmd_list.append(f"{bval.file}")
        mr_convert.cmd_list.append(f"{mif_file.file}")
        
        [return_code, stdout, stderr] = mr_convert.run(self.log)
        return mif_file
    
    def estimate_response(mif,method="dhollander"):
        '''doc-string'''
    
    def mr_upsample(mif,output):
        '''doc-string'''
    
    def create_mask(mif,frac_int):
        '''doc-string'''
        
    def ss3t_csd(mif,mask,wm_res,gm_res,csf_res):
        '''doc-string'''
    
    def bias_field_correction(wm_fod,gm,csf,mask):
        '''doc-string'''
    
    def compute_dec_map(wm_fod,mask):
        '''doc-string'''
    
    def mr_trk_global(wm_fod,mask,stream_lines, cutoff):
        '''doc-string'''
    
    # write visualization functions later

In [3]:
class DTKrecon(object):
    '''class doc-string'''
    
    def __init__(self):
        '''doc-string'''
    
    def dti_fit(dwi,mask,bvec,bval,threshold):
        '''doc-string'''
    
    def create_mask(dwi,bval):
        '''doc-string'''
    
    def extract_b0s(dwi,bval):
        '''doc-string'''
        # DWIxfm.extract_b0s
    
    def dti_recon(dwi,mask,threshold):
        '''doc-string'''
    
    # write visualization functions later

In [4]:
class DWIxfm(object):
    '''class doc-string'''
    
    def __init__(self):
        '''doc-string'''
    
    def extract_b0s(dwi,bval):
        '''doc-string'''
    
    def create_mask(dwi,bval):
        '''doc-string'''
    
    def compute_linear_xfm(dwi,template,dof):
        '''doc-string'''
    
    def compute_non_linear_xfm():
        '''doc-string'''
    
    def compute_combined_xfm():
        '''doc-string:
        computes combined linear and non-linear transform.'''
    
    def applywarp():
        '''doc-string'''

In [7]:
def test(file: File):
    pass